<a href="https://colab.research.google.com/github/cyberneel/ImageSecretEmbed/blob/main/ImageEmbedSecret.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Libraries

In [1]:
!pip install pillow numpy

# Main Code

In [4]:
from PIL import Image
import numpy as np
import io
from google.colab import files

def ToBinary(text):
    binary = ''.join(format(ord(char), '08b') for char in text)
    return binary

def ToText(binary):
    characters = [binary[i:i+8] for i in range(0, len(binary), 8)]
    textForm = ''.join([chr(int(char, 2)) for char in characters])
    return textForm

# Used to test proper conversion
print(ToBinary("Hello"))
print(ToText(ToBinary("Hello")))